In [11]:
from selenium.webdriver.support.ui import Select
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
def getICO(page_url, domain_name, limit=20):
    
    def getpage(page_url):
    
        page = requests.get(page_url)
        bs = BeautifulSoup(page.content, 'html.parser')
        base1 = bs.select('a.name.notranslate')
        base2 = bs.select('p.notranslate')
        base3 = bs.select('td.rmv')
        base4 = bs.select('div.rate')

        list_name = list(map(lambda x: re.findall(r'>(\S[^><]+)<', str(x))[0], base1))
        list_url = list(map(lambda x: re.findall(r'href="(/ico/[^ <>]+)">', str(x))[0], base1))
        list_desc1 = list(map(lambda x: re.findall(r'>([^ ][^<>]+)<br/>', str(x))[0], base2))
        list_desc2 = list(map(lambda x: re.findall(r'(Restrictions KYC:)</b>([^<>]+)', str(x))[0], base2))
        list_desc3 = list(map(lambda x: re.findall(r'(Whitelist:)</b>([^<>]+)', str(x))[0], base2))
        list_desc4 = list(map(lambda x: re.findall(r'(Countries:)</b>([^<>]+)', str(x)), base2))
        list_date = list(map(lambda x: re.findall(r'>([^<>]+)</td>', str(x)), base3))
        list_rating0 = list(map(lambda x: re.findall(r'>(.+)</div>', str(x))[0], base4))

        for i in range(len(base2)):
            if list_desc4[i] == []:
                list_desc4[i] = [('Countries:', ' Unknown')]    

        list_start = []
        list_end = []
        for i in range(len(base2)):
            list_start.append(list_date[3*i][0])    
            list_end.append(list_date[3*i+1][0])

        list_rating = []
        for i in range(len(base2)):
            list_rating.append(list_rating0[2*i])

        list_desc = list(map(lambda a, b, c, d: ' | '.join([a, b[0]+b[1], \
                        c[0]+c[1], d[0][0]+d[0][1]]), list_desc1, list_desc2, list_desc3, list_desc4))

        page_result = pd.DataFrame([list_name, list_url, list_desc, list_start, \
                                    list_end, list_rating], index=['name', 'url', 'desc', 'start', 'end', 'rating']).T

        return page_result
   
    ICO = getpage(page_url)
    current_url = page_url
    
    while len(ICO) < limit:
        page = requests.get(current_url)
        bs = BeautifulSoup(page.content, 'html.parser')
        next_ = bs.select('a.next')
        add_ = re.findall(r'href="([^<>]+)">', str(next_[0]))[0]
        current_url = domain_name + add_
        ICO_next = getpage(current_url)
        ICO = pd.concat([ICO, ICO_next]).reset_index(drop=True)

    return ICO

In [4]:
def getICOByCategory(driver, page_url, domain_name, category='Banking', limit=20):

    results = None
    
    driver.maximize_window()
    driver.get(page_url)

    select_category = driver.find_element_by_css_selector("select[name='filterCategory'][id='filterCategory']")
    Select(select_category).select_by_visible_text(category)

    time.sleep(5)

    sel1 = driver.find_elements_by_css_selector("a.name.notranslate")
    sel2 = driver.find_elements_by_css_selector('p.notranslate')
    sel3 = driver.find_elements_by_css_selector('td.rmv')
    sel4 = driver.find_elements_by_css_selector('div.rate')

    name_lis = list(map(lambda x: x.text, sel1))
    url_lis = list(map(lambda x: x.get_property('href')[len(domain_name):], sel1))
    desc_lis = list(map(lambda x: x.text.replace('\n\n', ' | '), sel2))
    date_lis = list(map(lambda x: x.text, sel3))

    start_lis = []
    end_lis = []
    rating_lis = []
    for i in range(len(sel1)):
        start_lis.append(date_lis[3*i])    
        end_lis.append(date_lis[3*i+1])
        rating_lis.append(date_lis[3*i+2])

    sel_result = pd.DataFrame([name_lis, url_lis, desc_lis, start_lis, \
                                end_lis, rating_lis], index=['name', 'url', 'desc', 'start', 'end', 'rating']).T    

    while len(sel_result) < limit:

        driver.find_element_by_css_selector('a.next').click()
        time.sleep(5)

        sel1 = driver.find_elements_by_css_selector("a.name.notranslate")
        sel2 = driver.find_elements_by_css_selector('p.notranslate')
        sel3 = driver.find_elements_by_css_selector('td.rmv')
        sel4 = driver.find_elements_by_css_selector('div.rate')

        name_lis = list(map(lambda x: x.text, sel1))
        url_lis = list(map(lambda x: x.get_property('href')[len(domain_name):], sel1))
        desc_lis = list(map(lambda x: x.text.replace('\n\n', ' | '), sel2))
        date_lis = list(map(lambda x: x.text, sel3))

        start_lis = []
        end_lis = []
        rating_lis = []
        for i in range(len(sel1)):
            start_lis.append(date_lis[3*i])    
            end_lis.append(date_lis[3*i+1])
            rating_lis.append(date_lis[3*i+2])

        sel_result_next = pd.DataFrame([name_lis, url_lis, desc_lis, start_lis, \
                                    end_lis, rating_lis], index=['name', 'url', 'desc', 'start', 'end', 'rating']).T 

        sel_result = pd.concat([sel_result, sel_result_next]).reset_index(drop=True)
        results = sel_result

    return results

In [12]:
if __name__ == "__main__":

    print("========Test Q1=========")
    domain_name = 'https://icobench.com'
    page_url = 'https://icobench.com/icos?'

    products = getICO(page_url, domain_name, limit=20)
    print(len(products))
    print(products.head(5))

    print("\n")
    print("========Test Q2=========")
    executable_path = 'D:/Stevens/Semester 2/Web Mining/Python courses/Web_Scraping/driver/msedgedriver'
    driver = webdriver.Edge(executable_path=executable_path)

    driver.implicitly_wait(10)  # set implict wait

    domain_name = 'https://icobench.com'
    page_url = 'https://icobench.com/icos?'


    products = getICOByCategory(driver, page_url, domain_name, category='Energy', limit=20)

    driver.quit()
    print(products)

========Test Q1=========
24


,name,url,desc,start,end,rating
0,Tycoon,/ico/tycoon,Tycoon serves as a secured interface between p...,01 Aug 2020,31 Mar 2021,5.0
1,Hashbon,/ico/hashbon,Hashbon framework - is a universal platform fo...,Unknown,Unknown,4.4
2,Emanate,/ico/emanate,The Emanate Audio Exchange Protocol is a decen...,Unknown,Unknown,4.1
3,Attrace,/ico/attrace,Attrace is the first fully decentralised affil...,Unknown,Unknown,4.0
4,MetaHash,/ico/metahash,#MetaHash is a next-generation network based o...,29 Jun 2018,Unknown,4.3




========Test Q2=========


C:\Users\lenovo\AppData\Local\Temp/ipykernel_19492/863441576.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path=executable_path)
C:\Users\lenovo\AppData\Local\Temp/ipykernel_19492/3256600730.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  select_category = driver.find_element_by_css_selector("select[name='filterCategory'][id='filterCategory']")
C:\Users\lenovo\AppData\Local\Temp/ipykernel_19492/3256600730.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  sel1 = driver.find_elements_by_css_selector("a.name.notranslate")
C:\Users\lenovo\AppData\Local\Temp/ipykernel_19492/3256600730.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  sel2 = driver.find_elements_by_css_selector('p.notranslate')
C:\Users\lenovo\AppData\Local\Temp/ipykern

,name,url,desc,start,end,rating
0,REDI,/ico/redi,A blockchain-based integrative data management...,Unknown,Unknown,3.9
1,serenitysource,/ico/serenitysource,Serenity - energy on the GO! Serenity is a nex...,01 Sep 2019,Unknown,3.8
2,MADANA,/ico/madana,MADANA is a platform for data analysis that us...,Unknown,Unknown,3.7
3,Nafen,/ico/nafen,Nafen nanofibers are able to change the mechan...,Unknown,Unknown,4.2
4,Verv (VLUX token),/ico/vlux,"Using AI and Blockchain technology, VLUX's pee...",04 Jul 2018,Unknown,3.9
5,GPM Carbon,/ico/gpm-carbon,GPM Carbon is a part of GPM Planet group and o...,16 Jul 2019,Unknown,3.5
6,Zero Carbon Project,/ico/zero-carbon-project,Our Zero Carbon Market beats renewable energy ...,Unknown,Unknown,4.1
7,Bitminer Factory,/ico/bitminer-factory,Feeding the Blockchain Revolution with renewab...,Unknown,Unknown,3.2
8,Irene Energy,/ico/irene-energy,Powering a sustainable future with the world's...,Unknown,Unknown,3.2
9,Eloncity,/ico/eloncity,BlockChain based trade systems make energy tra...,Unknown,Unknown,2.9
